In [7]:
!pip install -r requirements.txt
!pip install plotly
!pip install cufflinks

import ndb
from api import *
import warnings

import pandas as pd
import numpy as np
from scipy.optimize import linprog as lp

#usda api username and apikey
usda_key = "YeOlqLspekK5xbTISWB2j6mqtJMCEklBHYocrjj5"
usda_user = "sannidhishukla@berkeley.edu"

usdakey = {usda_user:usda_key}

In [4]:
foods = pd.read_csv('us_foods.csv', converters={'ndb': lambda x: str(x)})
foods['qty_purchased'] = [float(i) for i in foods['qty_purchased']]
foods.head()

,food,category,quantity,units,price,ndb,expend,qty_purchased
0,Beer,alcbev,12.0,oz,10.99,14004,1.614129,1.762470
1,Liquor and cocktails,alcbev,12.0,oz,10.99,14252,1.614129,1.762470
2,Wine,alcbev,750.0,ml,9.59,14253,1.614129,126.235315
3,Cookies and brownies,bakery,14.3,oz,5.69,45260051,2.635406,6.623252
4,Tortillas,bakery,20.0,oz,4.56,45365414,2.635406,11.558798


# Part Two: Compiling Nutrition Data

Next, we use the USDA NDB api to find the nutrition information on each of these foods. Because the NDB nutrition information uses very particular units (mainly grams and milliliters), we must also convert the units in our list of foods to match.

In [5]:
warnings.filterwarnings('ignore')

#use api calls to compile the nutritional information on different foods into a dataframe
nutrition = {}
for food in foods.food.tolist():
    try:
        NDB = foods.loc[foods.food==food,:].ndb
        nutrition[food] = ndb.ndb_report(usdakey[usda_user],NDB).Quantity
    except AttributeError: 
        warnings.warn("Couldn't find NDB Code %s for food %s." % (food,NDB))
        
nutrition = pd.DataFrame(nutrition,dtype=float).fillna(0)

# Convert food quantities to NDB units
foods['ndb_qty'] = foods[['qty_purchased','units']].T.apply(lambda x : ndb.ndb_units(x['qty_purchased'],x['units']))

nutrition.to_csv('nutrition.csv')

In [6]:
nutrition

,Beer,Liquor and cocktails,Wine,Cookies and brownies,Tortillas,Yeast breads,"Beef, excludes ground",Burgers,Ground beef,Oatmeal,...,Processed soy products,Apple juice,Citrus juice,Nutrition bars,Fish,Seafood mixed dishes,Shellfish,Candy containing chocolate,Ice cream and frozen dairy desserts,Sugars and honey
Caffeine,0.00,0.000,0.000,0.00,0.000,0.000,0.00,0.00,0.00,0.00,...,0.00,0.00,0.000,0.00,0.00,0.00,0.00,0.00,0.000,0.000
"Calcium, Ca",4.00,3.000,10.000,0.00,26.000,0.000,0.00,13.00,0.00,67.00,...,453.00,17.00,146.000,333.00,20.00,24.00,53.00,91.00,128.000,83.000
"Carbohydrate, by difference",2.97,0.000,3.800,72.41,46.510,42.860,3.57,0.00,0.00,63.33,...,52.20,46.67,10.830,75.00,3.57,0.00,0.88,68.18,23.600,98.090
Cholesterol,0.00,0.000,0.000,0.00,0.000,0.000,58.00,66.00,67.00,0.00,...,0.00,0.00,0.000,0.00,46.00,165.00,137.00,23.00,44.000,0.000
Energy,41.00,40.000,83.000,483.00,279.000,357.000,196.00,285.00,179.00,367.00,...,508.00,183.00,46.000,417.00,193.00,71.00,88.00,500.00,207.000,380.000
"Fatty acids, total monounsaturated",0.00,0.000,0.000,0.00,3.490,5.360,0.00,0.00,0.00,0.00,...,10.78,0.00,0.000,2.08,0.00,0.00,0.00,0.00,2.969,0.000
"Fatty acids, total polyunsaturated",0.00,0.000,0.000,0.00,1.160,0.000,0.00,0.00,0.00,0.00,...,3.83,0.00,0.000,0.00,0.00,0.00,0.00,0.00,0.452,0.000
"Fatty acids, total saturated",0.00,0.000,0.000,6.90,0.000,7.140,4.46,9.27,4.02,1.67,...,12.37,0.00,0.000,4.17,2.50,0.59,0.00,13.64,6.790,0.000
"Fatty acids, total trans",0.00,0.000,0.000,0.00,0.000,0.000,0.00,1.32,0.00,0.00,...,0.00,0.00,0.000,0.00,0.00,0.00,0.00,0.00,0.000,0.000
"Fiber, total dietary",0.00,0.000,0.000,3.40,7.000,0.000,0.00,0.00,0.00,10.00,...,0.00,0.00,0.000,4.20,0.20,0.00,0.00,0.00,0.700,0.000
